
**1. 구매 행렬표 생성:**

| 사용자 | 가위손 | 나 홀로 집에 | 다이하드 | 블레이드 러너 | 아마데우스 | 죽은 시인의 사회 |
|---|---|---|---|---|---|---|
| 1 | True | False | True | True | False | False |
| 2 | False | False | False | True | True | True |
| 3 | True | True | False | False | False | True |
| 4 | False | False | True | True | False | False |
| 5 | False | False | False | False | True | True |

**2. 1개 영화에 대한 지지도 계산:**

각 영화의 지지도를 계산 
  > 지지도:  전체 사용자 수(5명) 중 해당 영화를 구매한 사용자 수의 비율

*   가위손: 2/5 = 0.4
*   나 홀로 집에: 1/5 = 0.2
*   다이하드: 2/5 = 0.4
*   블레이드 러너: 3/5 = 0.6
*   아마데우스: 2/5 = 0.4
*   죽은 시인의 사회: 3/5 = 0.6

**3. 지지도가 가장 낮은 영화 제외:**

가장 낮은 지지도를 가진 영화는 "나 홀로 집에"(0.2)입니다. 이 영화를 제외합니다.

**4. 남은 영화로 2개씩 짝을 지어 지지도 계산:**

남은 영화들로 가능한 모든 2개 조합을 만들고 지지도를 계산

*   (가위손, 다이하드): 1/5 = 0.2
*   (가위손, 블레이드 러너): 1/5 = 0.2
*   (가위손, 아마데우스): 0/5 = 0
*   (가위손, 죽은 시인의 사회): 1/5 = 0.2
*   (다이하드, 블레이드 러너): 2/5 = 0.4
*   (다이하드, 아마데우스): 0/5 = 0
*   (다이하드, 죽은 시인의 사회): 0/5 = 0
*   (블레이드 러너, 아마데우스): 1/5 = 0.2
*   (블레이드 러너, 죽은 시인의 사회): 2/5 = 0.4
*   (아마데우스, 죽은 시인의 사회): 2/5 = 0.4

**5. 지지도가 가장 낮은 조합 제외:**

지지도가 0인 조합들과 지지도가 가장 낮은 0.2 지지도 조합을 제외

**6. 남은 조합으로 3개씩 짝을 지어 지지도 계산 (반복):**

남은 조합은 (다이하드, 블레이드 러너), (블레이드 러너, 죽은 시인의 사회), (아마데우스, 죽은 시인의 사회) 입니다. 이 조합들을 기반으로 3개 조합을 만들어야 하는데, 2개 조합만 남아있으므로 3개 조합은 만들 수 없습니다. 따라서 이 단계에서 종료됩니다.

**7. 최종 결과 및 해석:**

최소 지지도를 0.4로 가정했을 때, 남은 빈발 항목 집합은 다음과 같습니다.

*   {다이하드, 블레이드 러너}
*   {블레이드 러너, 죽은 시인의 사회}
*   {아마데우스, 죽은 시인의 사회}

이는 "다이하드와 블레이드 러너를 함께 본 사람", "블레이드 러너와 죽은 시인의 사회를 함께 본 사람", "아마데우스와 죽은 시인의 사회를 함께 본 사람"이 전체 사용자 중 40% 이상이라는 것을 의미합니다.


In [1]:
from itertools import combinations
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
data = {
    'user': [1, 2, 3, 4, 5],
    'movie_list': [
        ['가위손', '다이하드', '블레이드 러너'],
        ['블레이드 러너', '아마데우스', '죽은 시인의 사회'],
        ['가위손', '나 홀로 집에', '죽은 시인의 사회'],
        ['다이하드', '블레이드 러너'],
        ['죽은 시인의 사회', '아마데우스'],
    ]
}

df = pd.DataFrame(data)

In [3]:
# 트랜젝션 데이터(장바구니 분석, 연관 규칙 학습 등의 분야에서 데이터 전처리) 전치리용 도구
te = TransactionEncoder()

# numpy 형태로 반환한다.
te_ary = te.fit(df['movie_list']).transform(df['movie_list'])

df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

print("## 구매 행렬표 ##")
# print(te_ary)
print(df_encoded)

## 구매 행렬표 ##
     가위손  나 홀로 집에   다이하드  블레이드 러너  아마데우스  죽은 시인의 사회
0   True    False   True     True  False      False
1  False    False  False     True   True       True
2   True     True  False    False  False       True
3  False    False   True     True  False      False
4  False    False  False    False   True       True


In [4]:
min_support = 0.4
frequent_itemsets = apriori(df_encoded, min_support=min_support, use_colnames=True)

print("## 빈발 항목 집합 ##")
print(frequent_itemsets)

## 빈발 항목 집합 ##
   support            itemsets
0      0.4               (가위손)
1      0.4              (다이하드)
2      0.6           (블레이드 러너)
3      0.4             (아마데우스)
4      0.6         (죽은 시인의 사회)
5      0.4     (블레이드 러너, 다이하드)
6      0.4  (죽은 시인의 사회, 아마데우스)


In [5]:
rules = association_rules(frequent_itemsets, num_itemsets=len(frequent_itemsets), metric="confidence", min_threshold=0.5)
rules_lift = association_rules(frequent_itemsets, num_itemsets=len(frequent_itemsets), metric="lift", min_threshold=1)

print("## 연관 규칙 (신뢰도 기준) ##")
print(rules)
print("\n")

print("## 연관 규칙 (향상도 기준) ##")
print(rules_lift)
print("\n")

## 연관 규칙 (신뢰도 기준) ##
   antecedents  consequents  antecedent support  consequent support  support  \
0    (블레이드 러너)       (다이하드)                 0.6                 0.4      0.4   
1       (다이하드)    (블레이드 러너)                 0.4                 0.6      0.4   
2  (죽은 시인의 사회)      (아마데우스)                 0.6                 0.4      0.4   
3      (아마데우스)  (죽은 시인의 사회)                 0.4                 0.6      0.4   

   confidence      lift  representativity  leverage  conviction  \
0    0.666667  1.666667               1.0      0.16         1.8   
1    1.000000  1.666667               1.0      0.16         inf   
2    0.666667  1.666667               1.0      0.16         1.8   
3    1.000000  1.666667               1.0      0.16         inf   

   zhangs_metric   jaccard  certainty  kulczynski  
0       1.000000  0.666667   0.444444    0.833333  
1       0.666667  0.666667   1.000000    0.833333  
2       1.000000  0.666667   0.444444    0.833333  
3       0.666667  0.666667   1.000

In [6]:
num_transactions = len(df)

In [7]:
# 지지도 함수
def calc_support(itemset):
    count = 0
    for _, row in df_encoded.iterrows():
        if all(row[item] for item in itemset):
            count += 1
    return count / num_transactions

# 신뢰도 함수
def calc_confidence(antecedent, consequent):
    antecedent_support = calc_support(antecedent)

    if antecedent_support == 0:
        return 0
    joint_support = calc_support(antecedent + consequent)
    return joint_support / antecedent_support

# 향상도 함수
def calc_lift(antecedent, consequent):
    antecedent_support = calc_support(antecedent)
    consequent_support = calc_support(consequent)

    if antecedent_support == 0 or consequent_support == 0:
        return 0
    joint_support = calc_support(antecedent + consequent)
    return joint_support / (antecedent_support * consequent_support)

In [8]:
min_support = 0.4
freq_itemsets = []

for i in range(1, len(df_encoded.columns) + 1):
    for itemset in combinations(df_encoded.columns, i):
        support = calc_support(list(itemset))
        if support >= min_support:
            freq_itemsets.append((list(itemset), support))

print("## 빈발 항목 집합 ##")

for itemset, support in freq_itemsets:
    print(f"Itemset: {itemset}, Support: {support:.2f}")

## 빈발 항목 집합 ##
Itemset: ['가위손'], Support: 0.40
Itemset: ['다이하드'], Support: 0.40
Itemset: ['블레이드 러너'], Support: 0.60
Itemset: ['아마데우스'], Support: 0.40
Itemset: ['죽은 시인의 사회'], Support: 0.60
Itemset: ['다이하드', '블레이드 러너'], Support: 0.40
Itemset: ['아마데우스', '죽은 시인의 사회'], Support: 0.40


In [ ]:
association_rules = []
for itemset, _ in freq_itemsets:
    if len(itemset) >= 2:
        for i in range(1, len(itemset) + 1):
            for antecedent in combinations(itemset, i):
                consequent = list(set(itemset) - set(antecedent))
                confidence = calc_confidence(list(antecedent), consequent)
                lift = calc_lift(list(antecedent), consequent)
                association_rules.append((list(antecedent), consequent, confidence, lift))

print("## 연관 규칙 및 지표 ##")
for antecedent, consequent, confidence, lift in association_rules:
    support = calc_support(antecedent + consequent)
    print(f"규칙: {antecedent} -> {consequent}")
    print(f"  지지도: {support:.2f}")
    print(f"  신뢰도: {confidence:.2f}")
    print(f"  향상도: {lift:.2f}")
    print("-" * 20)

## 연관 규칙 및 지표 ##
규칙: ['다이하드'] -> ['블레이드 러너']
  지지도: 0.40
  신뢰도: 1.00
  향상도: 1.67
--------------------
규칙: ['블레이드 러너'] -> ['다이하드']
  지지도: 0.40
  신뢰도: 0.67
  향상도: 1.67
--------------------
규칙: ['아마데우스'] -> ['죽은 시인의 사회']
  지지도: 0.40
  신뢰도: 1.00
  향상도: 1.67
--------------------
규칙: ['죽은 시인의 사회'] -> ['아마데우스']
  지지도: 0.40
  신뢰도: 0.67
  향상도: 1.67
--------------------
